<a href="https://colab.research.google.com/github/KaavyaRekanar/Subjective_Scoring_Framework_VQA/blob/main/Case_Study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.8 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=fb20871c20948934d7568c606907fde55d9fddb0e7fd9920f2a076e5d7cf0859
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation, util,models
from torch.utils.data import DataLoader
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from torch import nn


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# Read and store content
# of an excel file
#read_file = pd.read_excel ("/content/drive/MyDrive/Subjective_scoring.xlsx")
read_file = pd.read_excel ("/content/drive/MyDrive/case_study.xlsx")

# Write the dataframe object
# into csv file
read_file.to_csv ("/content/drive/MyDrive/VQA.csv",
                   index = None,
                header=True)

# read csv file and convert
# into a dataframe object
df = pd.DataFrame(pd.read_csv("/content/drive/MyDrive/VQA.csv"))

# show the dataframe
df

,Category,Questions,Answers,ViLBERT,ViLBERT_Score,ViLT,ViLT_Score,LXMERT,LXMERT_Score
0,Dark setting,what are the contents of the image?,An Accident,Trucks,0,Cars,0.00,Cars,0.0
1,NaN,what should the driver do?,Slow down and go left,Run,0,Stop,0.00,Go,0.5
2,Light Setting,what are the contents of the image?,Traffic Lights,Clouds,0,Traffic Lights,1.00,Power Lines,0.0
3,NaN,what should the driver do?,Stop,Sleep,0,Stop,1.00,Stop,1.0
4,Parking,what are the contents of the image?,Parking Lot,Clouds,0,Cars,0.00,Cars,0.0
5,NaN,Can the driver park here?,Yes,No,0,No,0.00,Yes,1.0
6,Signboard,what are the contents of the image?,Road,Paint,0,Traffic Lights,0.00,Cars,0.0
7,NaN,what should the driver do?,Stop,Sleep,0,Stop,0.00,Go,0.0
8,Pedestrian Crossing,what are the contents of the image?,People crossing the road,Clouds,0,Buildings,0.00,People,1.0
9,NaN,what should the driver do?,Stop,Run,0,Stop,1.00,Stop,1.0


##BERT based Models

In [ ]:
model1 =SentenceTransformer('nli-distilbert-base')

## Function for Embedding Generation and Cosine Similarity

In [ ]:
def pred(model,q_s, ans):
    try:
      return util.cos_sim(model.encode(q_s), model.encode(ans))[0].cpu().item()
    except:
      return 0.0

In [ ]:
df['predict_nli_ViLT'] = df.apply(lambda row: pred(model1, row['Answers'],row['ViLT']), axis=1)

df['predict_nli_LXMERT'] = df.apply(lambda row: pred(model1, row['Answers'],row['LXMERT']), axis=1)

df['predict_nli_ViLBERT'] = df.apply(lambda row: pred(model1, row['Answers'],row['ViLBERT']), axis=1)

In [ ]:
df

,Category,Questions,Ground Truth,ViLBERT,ViLBERT Score,ViLT,ViLT Score,LXMERT,LXMERT Score,predict_nli_ViLT,predict_nli_LXMERT,predict_nli_ViLBERT
0,Dark setting,what are the contents of the image?,An Accident,Trucks,0,Cars,0.00,Cars,0.0,0.514676,0.514676,0.484044
1,NaN,what should the driver do?,Slow down and go left,Run,0,Stop,0.00,Go,0.5,0.820524,0.745009,0.622855
2,Light Setting,what are the contents of the image?,Traffic Lights,Clouds,0,Traffic Lights,1.00,Power Lines,NaN,1.000000,0.529374,0.634766
3,NaN,what should the driver do?,Stop,Sleep,0,Stop,1.00,Stop,1.0,1.000000,1.000000,0.675775
4,Parking,what are the contents of the image?,Parking Lot,Clouds,0,Cars,0.00,Cars,0.0,0.775691,0.775691,0.513588
5,NaN,Can the driver park here?,Yes,No,0,No,0.00,Yes,1.0,0.763509,1.000000,0.763509
6,Signboard,what are the contents of the image?,Road,Paint,0,Traffic Lights,0.00,Cars,0.0,0.745302,0.751782,0.539068
7,NaN,what should the driver do?,Stop,Sleep,0,Stop,0.00,Go,0.0,1.000000,0.832660,0.675775
8,Pedestrian Crossing,what are the contents of the image?,People crossing the road,Clouds,0,Buildings,0.00,People,1.0,0.611355,0.597221,0.576276
9,NaN,what should the driver do?,Stop,Run,0,Stop,1.00,Stop,1.0,1.000000,1.000000,0.616396


In [ ]:
def pearson_correlation(x,y):
    mean_x = sum(x) / len(x)
    mean_y = sum(y) / len(y)
    cov = sum((a - mean_x) * (b - mean_y) for (a, b) in zip(x, y)) / len(x)

    std_x, std_y = np.std(x), np.std(y)

    p = cov / (std_x * std_y)

    return float(p)

In [ ]:
print("Pearson Correlation for ViLT:", pearson_correlation(df['ViLT_Score'], df['predict_nli_ViLT']))
print("Pearson Correlation for LXMERT:", pearson_correlation(df['LXMERT_Score'], df['predict_nli_LXMERT']))
print("Pearson Correlation for ViLBERT:", pearson_correlation(df['ViLBERT_Score'], df['predict_nli_ViLBERT']))

Pearson Correlation for ViLT: 0.6494520524780797
Pearson Correlation for LXMERT: 0.6587834069393875
Pearson Correlation for ViLBERT: 0.7686468101272999


In [ ]:
model2= SentenceTransformer('all-mpnet-base-v2') #This one is identified as one of the best performing model in SBERT documentation

In [ ]:
df['predict_mpnet_ViLT'] = df.apply(lambda row: pred(model2, row['Answers'],row['ViLT']), axis=1)

df['predict_mpnet_LXMERT'] = df.apply(lambda row: pred(model2, row['Answers'],row['LXMERT']), axis=1)

df['predict_mpnet_ViLBERT'] = df.apply(lambda row: pred(model2, row['Answers'],row['ViLBERT']), axis=1)

In [ ]:
df

,Category,Questions,Answers,ViLBERT,ViLBERT_Score,ViLT,ViLT_Score,LXMERT,LXMERT_Score,predict_nli_ViLT,predict_nli_LXMERT,predict_nli_ViLBERT,predict_mpnet_ViLT,predict_mpnet_LXMERT,predict_mpnet_ViLBERT
0,Dark setting,what are the contents of the image?,An Accident,Trucks,0,Cars,0.00,Cars,0.0,0.514676,0.514676,0.484044,0.336592,0.336592,0.287782
1,NaN,what should the driver do?,Slow down and go left,Run,0,Stop,0.00,Go,0.5,0.820524,0.745009,0.622855,0.341135,0.279464,0.319637
2,Light Setting,what are the contents of the image?,Traffic Lights,Clouds,0,Traffic Lights,1.00,Power Lines,0.0,1.000000,0.529374,0.634766,1.000000,0.350515,0.330589
3,NaN,what should the driver do?,Stop,Sleep,0,Stop,1.00,Stop,1.0,1.000000,1.000000,0.675775,1.000000,1.000000,0.300823
4,Parking,what are the contents of the image?,Parking Lot,Clouds,0,Cars,0.00,Cars,0.0,0.775691,0.775691,0.513588,0.434045,0.434045,0.292942
5,NaN,Can the driver park here?,Yes,No,0,No,0.00,Yes,1.0,0.763509,1.000000,0.763509,0.700037,1.000000,0.700037
6,Signboard,what are the contents of the image?,Road,Paint,0,Traffic Lights,0.00,Cars,0.0,0.745302,0.751782,0.539068,0.385419,0.476891,0.225300
7,NaN,what should the driver do?,Stop,Sleep,0,Stop,0.00,Go,0.0,1.000000,0.832660,0.675775,1.000000,0.286398,0.300823
8,Pedestrian Crossing,what are the contents of the image?,People crossing the road,Clouds,0,Buildings,0.00,People,1.0,0.611355,0.597221,0.576276,0.062371,0.304802,0.032645
9,NaN,what should the driver do?,Stop,Run,0,Stop,1.00,Stop,1.0,1.000000,1.000000,0.616396,1.000000,1.000000,0.327743


In [ ]:
print("Pearson Correlation for ViLT:", pearson_correlation(df['ViLT_Score'], df['predict_mpnet_ViLT']))
print("Pearson Correlation for LXMERT:", pearson_correlation(df['LXMERT_Score'], df['predict_mpnet_LXMERT']))
print("Pearson Correlation for ViLBERT:", pearson_correlation(df['ViLBERT_Score'], df['predict_mpnet_ViLBERT']))

Pearson Correlation for ViLT: 0.7446737142817109
Pearson Correlation for LXMERT: 0.8215087753052542
Pearson Correlation for ViLBERT: 0.8286081771849688


##Basic BERT Model

In [ ]:
word_embedding_model = models.Transformer('bert-base-uncased', max_seq_length=256)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
dense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), out_features=256, activation_function=nn.Tanh())

model3 = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])

In [ ]:
df['bert_ViLT'] = df.apply(lambda row: pred(model3, row['Answers'],row['ViLT']), axis=1)

df['bert_LXMERT'] = df.apply(lambda row: pred(model3, row['Answers'],row['LXMERT']), axis=1)

df['bert_ViLBERT'] = df.apply(lambda row: pred(model3, row['Answers'],row['ViLBERT']), axis=1)

In [ ]:
df

,Category,Questions,Answers,ViLBERT,ViLBERT_Score,ViLT,ViLT_Score,LXMERT,LXMERT_Score,predict_nli_ViLT,predict_nli_LXMERT,predict_nli_ViLBERT,predict_mpnet_ViLT,predict_mpnet_LXMERT,predict_mpnet_ViLBERT,bert_ViLT,bert_LXMERT,bert_ViLBERT
0,Dark setting,what are the contents of the image?,An Accident,Trucks,0,Cars,0.00,Cars,0.0,0.514676,0.514676,0.484044,0.336592,0.336592,0.287782,0.741210,0.741210,0.735952
1,NaN,what should the driver do?,Slow down and go left,Run,0,Stop,0.00,Go,0.5,0.820524,0.745009,0.622855,0.341135,0.279464,0.319637,0.452840,0.459277,0.531457
2,Light Setting,what are the contents of the image?,Traffic Lights,Clouds,0,Traffic Lights,1.00,Power Lines,0.0,1.000000,0.529374,0.634766,1.000000,0.350515,0.330589,1.000000,0.760997,0.655872
3,NaN,what should the driver do?,Stop,Sleep,0,Stop,1.00,Stop,1.0,1.000000,1.000000,0.675775,1.000000,1.000000,0.300823,1.000000,1.000000,0.722997
4,Parking,what are the contents of the image?,Parking Lot,Clouds,0,Cars,0.00,Cars,0.0,0.775691,0.775691,0.513588,0.434045,0.434045,0.292942,0.642632,0.642632,0.617960
5,NaN,Can the driver park here?,Yes,No,0,No,0.00,Yes,1.0,0.763509,1.000000,0.763509,0.700037,1.000000,0.700037,0.850164,1.000000,0.850164
6,Signboard,what are the contents of the image?,Road,Paint,0,Traffic Lights,0.00,Cars,0.0,0.745302,0.751782,0.539068,0.385419,0.476891,0.225300,0.628178,0.796715,0.765733
7,NaN,what should the driver do?,Stop,Sleep,0,Stop,0.00,Go,0.0,1.000000,0.832660,0.675775,1.000000,0.286398,0.300823,1.000000,0.887554,0.722997
8,Pedestrian Crossing,what are the contents of the image?,People crossing the road,Clouds,0,Buildings,0.00,People,1.0,0.611355,0.597221,0.576276,0.062371,0.304802,0.032645,0.626494,0.575549,0.590055
9,NaN,what should the driver do?,Stop,Run,0,Stop,1.00,Stop,1.0,1.000000,1.000000,0.616396,1.000000,1.000000,0.327743,1.000000,1.000000,0.813360


In [ ]:
print("Pearson Correlation for ViLT:", pearson_correlation(df['ViLT_Score'], df['bert_ViLT']))
print("Pearson Correlation for LXMERT:", pearson_correlation(df['LXMERT_Score'], df['bert_LXMERT']))
print("Pearson Correlation for ViLBERT:", pearson_correlation(df['ViLBERT_Score'], df['bert_ViLBERT']))

Pearson Correlation for ViLT: 0.6109431917958806
Pearson Correlation for LXMERT: 0.5183189020190019
Pearson Correlation for ViLBERT: 0.7076686711437513


##GPT2 Model -- GPT3 is not freely available

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.4 MB/s eta 0:00:00


In [ ]:
import torch
import pandas as pd
from transformers import GPT2Tokenizer, GPT2Model
from sklearn.metrics.pairwise import cosine_similarity

# Load the pre-trained GPT-2 model and tokenizer
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2Model.from_pretrained(model_name)


# Generate embeddings for the text pairs in the DataFrame
embeddings_list = []

for _, row in df.iterrows():
    text1 = row['Answers']
    text2 = row['ViLT']

    # Tokenize and encode the texts using the GPT-2 tokenizer
    inputs1 = tokenizer(text1, return_tensors='pt', padding=False, truncation=True)
    inputs2 = tokenizer(text2, return_tensors='pt', padding=False, truncation=True)

    # Generate embeddings for the texts using GPT-2
    with torch.no_grad():
        embeddings1 = model(**inputs1).last_hidden_state.mean(dim=1)
        embeddings2 = model(**inputs2).last_hidden_state.mean(dim=1)

    embeddings_list.append((embeddings1, embeddings2))

# Add embeddings to the DataFrame
df['embeddings1'], df['embeddings2'] = zip(*embeddings_list)

# Calculate cosine similarity between the embeddings and store it in a new column
cosine_similarities = []

for _, row in df.iterrows():
    similarity = cosine_similarity(row['embeddings1'].numpy(), row['embeddings2'].numpy())
    cosine_similarities.append(similarity[0][0])

df.drop(columns=['embeddings1', 'embeddings2'], inplace=True)

df['gpt2_ViLT'] = cosine_similarities

# The DataFrame now contains text embeddings and cosine similarity values




In [ ]:
embeddings_list1 = []

for _, row in df.iterrows():
    text1 = row['Answers']
    text2 = row['LXMERT']

    # Tokenize and encode the texts using the GPT-2 tokenizer
    inputs1 = tokenizer(text1, return_tensors='pt', padding=False, truncation=True)
    inputs2 = tokenizer(text2, return_tensors='pt', padding=False, truncation=True)

    # Generate embeddings for the texts using GPT-2
    with torch.no_grad():
        embeddings1 = model(**inputs1).last_hidden_state.mean(dim=1)
        embeddings2 = model(**inputs2).last_hidden_state.mean(dim=1)

    embeddings_list1.append((embeddings1, embeddings2))

# Add embeddings to the DataFrame
df['embeddings1'], df['embeddings2'] = zip(*embeddings_list1)

# Calculate cosine similarity between the embeddings and store it in a new column
cosine_similarities1 = []

for _, row in df.iterrows():
    similarity = cosine_similarity(row['embeddings1'].numpy(), row['embeddings2'].numpy())
    cosine_similarities1.append(similarity[0][0])

df.drop(columns=['embeddings1', 'embeddings2'], inplace=True)

df['gpt2_LXMERT'] = cosine_similarities1

# The DataFrame now contains text embeddings and cosine similarity values


In [ ]:
embeddings_list2 = []

for _, row in df.iterrows():
    text1 = row['Answers']
    text2 = row['ViLBERT']

    # Tokenize and encode the texts using the GPT-2 tokenizer
    inputs1 = tokenizer(text1, return_tensors='pt', padding=False, truncation=True)
    inputs2 = tokenizer(text2, return_tensors='pt', padding=False, truncation=True)

    # Generate embeddings for the texts using GPT-2
    with torch.no_grad():
        embeddings1 = model(**inputs1).last_hidden_state.mean(dim=1)
        embeddings2 = model(**inputs2).last_hidden_state.mean(dim=1)

    embeddings_list2.append((embeddings1, embeddings2))

# Add embeddings to the DataFrame
df['embeddings1'], df['embeddings2'] = zip(*embeddings_list2)

# Calculate cosine similarity between the embeddings and store it in a new column
cosine_similarities2 = []

for _, row in df.iterrows():
    similarity = cosine_similarity(row['embeddings1'].numpy(), row['embeddings2'].numpy())
    cosine_similarities2.append(similarity[0][0])

df.drop(columns=['embeddings1', 'embeddings2'], inplace=True)

df['gpt2_ViLBERT'] = cosine_similarities2


In [ ]:
df


,Category,Questions,Answers,ViLBERT,ViLBERT_Score,ViLT,ViLT_Score,LXMERT,LXMERT_Score,predict_nli_ViLT,...,predict_nli_ViLBERT,predict_mpnet_ViLT,predict_mpnet_LXMERT,predict_mpnet_ViLBERT,bert_ViLT,bert_LXMERT,bert_ViLBERT,gpt2_ViLT,gpt2_LXMERT,gpt2_ViLBERT
0,Dark setting,what are the contents of the image?,An Accident,Trucks,0,Cars,0.00,Cars,0.0,0.514676,...,0.484044,0.336592,0.336592,0.287782,0.741210,0.741210,0.735952,0.996490,0.996490,0.994659
1,NaN,what should the driver do?,Slow down and go left,Run,0,Stop,0.00,Go,0.5,0.820524,...,0.622855,0.341135,0.279464,0.319637,0.452840,0.459277,0.531457,0.968816,0.960553,0.965148
2,Light Setting,what are the contents of the image?,Traffic Lights,Clouds,0,Traffic Lights,1.00,Power Lines,0.0,1.000000,...,0.634766,1.000000,0.350515,0.330589,1.000000,0.760997,0.655872,1.000000,0.998134,0.996017
3,NaN,what should the driver do?,Stop,Sleep,0,Stop,1.00,Stop,1.0,1.000000,...,0.675775,1.000000,1.000000,0.300823,1.000000,1.000000,0.722997,1.000000,1.000000,0.994700
4,Parking,what are the contents of the image?,Parking Lot,Clouds,0,Cars,0.00,Cars,0.0,0.775691,...,0.513588,0.434045,0.434045,0.292942,0.642632,0.642632,0.617960,0.990138,0.990138,0.994135
5,NaN,Can the driver park here?,Yes,No,0,No,0.00,Yes,1.0,0.763509,...,0.763509,0.700037,1.000000,0.700037,0.850164,1.000000,0.850164,0.992837,1.000000,0.951846
6,Signboard,what are the contents of the image?,Road,Paint,0,Traffic Lights,0.00,Cars,0.0,0.745302,...,0.539068,0.385419,0.476891,0.225300,0.628178,0.796715,0.765733,0.985569,0.990855,0.985829
7,NaN,what should the driver do?,Stop,Sleep,0,Stop,0.00,Go,0.0,1.000000,...,0.675775,1.000000,0.286398,0.300823,1.000000,0.887554,0.722997,1.000000,0.993942,0.994700
8,Pedestrian Crossing,what are the contents of the image?,People crossing the road,Clouds,0,Buildings,0.00,People,1.0,0.611355,...,0.576276,0.062371,0.304802,0.032645,0.626494,0.575549,0.590055,0.981600,0.979405,0.979031
9,NaN,what should the driver do?,Stop,Run,0,Stop,1.00,Stop,1.0,1.000000,...,0.616396,1.000000,1.000000,0.327743,1.000000,1.000000,0.813360,1.000000,1.000000,0.996244


In [ ]:
print("Pearson Correlation for ViLT:", pearson_correlation(df['ViLT_Score'], df['gpt2_ViLT']))
print("Pearson Correlation for LXMERT:", pearson_correlation(df['LXMERT_Score'], df['gpt2_LXMERT']))
print("Pearson Correlation for ViLBERT:", pearson_correlation(df['ViLBERT_Score'], df['gpt2_ViLBERT']))

Pearson Correlation for ViLT: 0.39034178436647293
Pearson Correlation for LXMERT: 0.19518511712539008
Pearson Correlation for ViLBERT: 0.2521560136791964


## Mean Absolute Error

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
print(" For NLI Model ")
print("MAE for ViLT:", mean_absolute_error(df['ViLT_Score'],df['predict_nli_ViLT']))
print("MAE for LXMERT:", mean_absolute_error(df['ViLT_Score'],df['predict_nli_LXMERT']))
print("MAE for ViLBERT:", mean_absolute_error(df['ViLBERT_Score'],df['predict_nli_ViLBERT']))

 For NLI Model 
MAE for ViLT: 0.49011569944295014
MAE for LXMERT: 0.524149635976011
MAE for ViLBERT: 0.558093400164084


In [ ]:
print(" For mpnet Model")
print("MAE for ViLT:", mean_absolute_error(df['ViLT_Score'],df['predict_mpnet_ViLT']))
print("MAE for LXMERT:", mean_absolute_error(df['ViLT_Score'],df['predict_mpnet_LXMERT']))
print("MAE for ViLBERT:", mean_absolute_error(df['ViLBERT_Score'],df['predict_mpnet_ViLBERT']))

 For mpnet Model
MAE for ViLT: 0.2955099100416357
MAE for LXMERT: 0.3960701450705528
MAE for ViLBERT: 0.28140803951431403


In [ ]:
print(" For BERT Model")
print("MAE for ViLT:", mean_absolute_error(df['ViLT_Score'],df['bert_ViLT']))
print("MAE for LXMERT:", mean_absolute_error(df['ViLT_Score'],df['bert_LXMERT']))
print("MAE for ViLBERT:", mean_absolute_error(df['ViLBERT_Score'],df['bert_ViLBERT']))

 For BERT Model
MAE for ViLT: 0.4937367520549081
MAE for LXMERT: 0.5124108791351318
MAE for ViLBERT: 0.6464045101946051


In [ ]:
print(" For GPT-2 Model")
print("MAE for ViLT:", mean_absolute_error(df['ViLT_Score'],df['gpt2_ViLT']))
print("MAE for LXMERT:", mean_absolute_error(df['ViLT_Score'],df['gpt2_LXMERT']))
print("MAE for ViLBERT:", mean_absolute_error(df['ViLBERT_Score'],df['gpt2_ViLBERT']))

 For GPT-2 Model
MAE for ViLT: 0.6631663550030101
MAE for LXMERT: 0.663283879106695
MAE for ViLBERT: 0.8998978435993195


## Root Mean Squared Error for All Models

In [ ]:
from sklearn.metrics import mean_squared_error
import math
#nli RMSE   "Sqaured= True (MSE) , squared= False (RMSE)"
print("RMSE ViLT",math.sqrt(mean_squared_error(df['ViLT_Score'],df['predict_nli_ViLT'], squared =False)))
print("RMSE LXMERT",math.sqrt(mean_squared_error(df['LXMERT_Score'],df['predict_nli_LXMERT'], squared =False)))
print("RMSE ViLBERT",math.sqrt(mean_squared_error(df['ViLBERT_Score'],df['predict_nli_ViLBERT'], squared =False)))

RMSE ViLT 0.7742616155469544
RMSE LXMERT 0.7427342639487056
RMSE ViLBERT 0.7697902009848183


In [ ]:
# mpnet RMSE
print("RMSE ViLT",math.sqrt(mean_squared_error(df['ViLT_Score'],df['predict_mpnet_ViLT'], squared =False)))
print("RMSE LXMERT",math.sqrt(mean_squared_error(df['LXMERT_Score'],df['predict_mpnet_LXMERT'], squared =False)))
print("RMSE ViLBERT",math.sqrt(mean_squared_error(df['ViLBERT_Score'],df['predict_mpnet_ViLBERT'], squared =False)))

RMSE ViLT 0.6241067985199525
RMSE LXMERT 0.5697198774188846
RMSE ViLBERT 0.5693696059081225


In [ ]:
##BERT RMSE
print("RMSE ViLT",math.sqrt(mean_squared_error(df['ViLT_Score'],df['bert_ViLT'], squared =False)))
print("RMSE LXMERT",math.sqrt(mean_squared_error(df['LXMERT_Score'],df['bert_LXMERT'], squared =False)))
print("RMSE ViLBERT",math.sqrt(mean_squared_error(df['ViLBERT_Score'],df['bert_ViLBERT'], squared =False)))

RMSE ViLT 0.7773646759158049
RMSE LXMERT 0.7776637145862153
RMSE ViLBERT 0.8264435296828151


In [ ]:
#GPT-2 RMSE
print("RMSE ViLT",math.sqrt(mean_squared_error(df['ViLT_Score'],df['gpt2_ViLT'], squared =False)))
print("RMSE LXMERT",math.sqrt(mean_squared_error(df['LXMERT_Score'],df['gpt2_LXMERT'], squared =False)))
print("RMSE ViLBERT",math.sqrt(mean_squared_error(df['ViLBERT_Score'],df['gpt2_ViLBERT'], squared =False)))

RMSE ViLT 0.8937145362891836
RMSE LXMERT 0.8929432233152603
RMSE ViLBERT 0.9715383351247422
